# Kmer Composition

kmers are a very useful thing in bioinformatics. Sepcially when it comes to genome assembly. The first step in a kmer analysis is usually finding what kmers are there. We have done this in almost every problem we've solved so far:

In [68]:
def kmer_composition(text, k):
    kmers = {}
    for i in range(len(text) - k + 1):
        kmer = text[i: i + k]
        kmers[kmer] = True
    # we have a dictionary that doesn't have duplicates
    kmer_list = []
    for kmer in kmers:
        kmer_list.append(kmer)
    return kmer_list

In [69]:
dna = 'ACTAGCTAGATCGA'

kmer_composition(dna, 3)

['ACT', 'CTA', 'TAG', 'AGC', 'GCT', 'AGA', 'GAT', 'ATC', 'TCG', 'CGA']

# String Spelled by a Genome Path Problem

Given a number of overlapping kmers, we want to rebuild the string they "spell".

Because each kmer share k - 1 bases with the previous one, each pair adds 1 base to this "assembly".

In [76]:
def spell_assembly(kmers):
    k = len(kmers[0])
    base = kmers[0]
    for kmer in kmers[1:]:
        base = base + kmer[-1]
    return base

In [77]:
kmers = \
['ACCGA', 
  'CCGAA',
   'CGAAG',
    'GAAGC',
     'AAGCT'
]


spell_assembly(kmers)

'ACCGAAGCT'

# Overlap Graph Problem

We create an adjaceny matrix for the overlap graph made out of a set of kmers. Two kmers overlap if one's suffix is the prefix of another one. 

Here we define suffix as the last k - 1 bases and the prefix as the first k - 1 bases. These can be more generally defined as the last or first $q$ bases where $1 \leq q \leq k - 1$.

There are two ways to store a graph. An adjaceny list or an adjacency matrix. 

The adjacency matrix will store the whol $n\times n$ matrix in memory (e.g a 2-dimensional array). This has the disadvantage that if the matrix is sparse (e.g two many zeros, nodes that aren't connected, we will be wasting space). Advantage is that we can check if two nodes $i$ and $j$ are connected in constant time, by looking at $a_{i,j}$.

An adjacency list representation instead store the "list" of nodes each node is connected to. This uses less memory in a proper implementaion but checking if a certain node is connected can take $O(n)$ times in a naive implementation because the list for the corresponding node has to be searched. (How to improve this? :D)

In [11]:
def create_overlap_graph(kmers):
    # create matrix
    a = []
    k = len(kmers[0])
    for i in range(k):
        a.append([])
        for j in range(k):
            a[i].append(0)
    # shorter version. Try using this, what goes wrong?
    #a = [[0] * len(kmers)] * len(kmers)
    for i, p in enumerate(kmers):
        for j, q in enumerate(kmers):
            if p[1:] == q[:k - 1]:
                a[i][j] = 1
    print(a)
    for i, p in enumerate(kmers):
        for j, q in enumerate(kmers):
            if a[i][j] == 1:
                print(p, '->', q)

In [13]:
kmers = [
    'ATGCG',
    'GCATG',
    'CATGC',
    'AGGCA',
    'GGCAT',
]

create_overlap_graph(kmers)

[[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0]]
GCATG -> CATGC
CATGC -> ATGCG
AGGCA -> GGCAT
GGCAT -> GCATG


As you can the matrix above is mostly zeros, meaning it's sparse. It's usualy the case the matrices for kmer overlap graphs are sparse (you can even prove that mathematically for a random set of kmers, maybe try if you're curious).

# De Bruijn Graph Construction

The Hamiltonian Path problem, if it could be solved efficiently, would be the ultimate solution for genome assembly. However this is "comptuationally hard" problem, meaning no algorithm exists that can solve it in reasonable time for large data. Instead we should try to solve approximation of the problem. One commone way to deal with this is to modify the graph in such a way that the edges of the new graph correspond to the nodes of the previous graph. The de Bruijn graph is an example of such a transformation.

In [90]:
def build_de_bruijn_graph(k, text):
    kmers = []
    for i in range(len(text) - k + 1):
        kmer = text[i: i + k]
        kmers.append(kmer)
    graph = {}
    for kmer in kmers:
        node = kmer[:k - 1]
        if not node in graph:
            graph[node] = []
        graph[node].append(kmer[1:])
    for p in graph:
        s = p + ' -> '
        for i, q in enumerate(graph[p]):
            s += q
            if i != len(graph[p]) - 1:
                s += ','
        print(s)

In [91]:
def build_de_bruijn_graph(k, text):
    kmers = []
    for i in range(len(text) - k + 1):
        kmer = text[i:i+k]
        kmers.append(kmer)
    matrix = {}
    for kmer in kmers:
        node = kmer[:k - 1]
        if not node in matrix:
            matrix[node] = []
        matrix[node].append(kmer[1:])

    for a, node in enumerate(matrix):
        n = str(a + 1) + ' '
        l = list(matrix)
        for i, j in enumerate(matrix[node]):
            if j in l:
                index_1 = str(l.index(j) + 1)
                print(n + index_1)
            else:
                pass

    # for j in matrix:
    #     n = j + ' -> '
    #     for a, b in enumerate(matrix[j]):
    #         n += b
    #         if a != len(matrix[j]) - 1:
    #             n += ','
    #     print(n)

In [92]:
build_de_bruijn_graph(4, 'AAGATTCTCTAC')



1 2
2 3
3 4
4 5
5 6
6 7
6 8
7 6


Note that in the previous example we knew the ordering of kmers in the DNA. In real world we won't know this information because the sequencing machines produces millions of reads in random order. What if we only have the kmers as input?

In [27]:
def build_de_bruijn_graph_from_kmers(kmers):
    matrix = {}
    k = len(kmers[0])
    for kmer in kmers:
        node = kmer[:k - 1]
        if not node in matrix:
            matrix[node] = []
        matrix[node].append(kmer[1:])
    for p in matrix:
        s = p + ' -> '
        for i, q in enumerate(matrix[p]):
            s += q
            if i != len(matrix[p]) - 1:
                s += ','
        print(s)

In [28]:
kmers = [
    'GAGG',
    'CAGG',
    'GGGG',
    'GGGA',
    'CAGG',
    'AGGG',
    'GGAG',
]

'GAG' -> [AGG]
'CAG' -> [AGG]
'GGG' -> [GGG, GGA]



build_de_bruijn_graph_from_kmers(kmers)

GAG -> AGG
CAG -> AGG,AGG
GGG -> GGG,GGA
AGG -> GGG
GGA -> GAG


# Finding Eulerian Cycles

We use Hierholzer's algorithm reproduced here from Wikipedia:

* Choose any starting vertex v, and follow a trail of edges from that vertex until returning to v. It is not possible to get stuck at any vertex other than v, because the even degree of all vertices ensures that, when the trail enters another vertex w there must be an unused edge leaving w. The tour formed in this way is a closed tour, but may not cover all the vertices and edges of the initial graph.
* As long as there exists a vertex u that belongs to the current tour but that has adjacent edges not part of the tour, start another trail from u, following unused edges until returning to u, and join the tour formed in this way to the previous tour.
* Since we assume the original graph is connected, repeating the previous step will exhaust all edges of the graph.

In [88]:
def find_cycle(graph, v):
    cycle = []
    while len(graph[v]) != 0:
        u = graph[v][0]
        cycle.append(u)
        graph[v].pop(0)
        v = u
    return cycle

def append_cycle(graph, cycle, node):
    path = []
    v = node
    while len(graph[v]) != 0:
        u = graph[v][0]
        path.append(u)
        graph[v].pop(0)
        if u in cycle:
            print(path)
            i = cycle.index(node) + 1
            for p in path:
                cycle.insert(i, p)
                i += 1
            break
        v = u
    print(cycle)
    return cycle

def eulerian_cycle(graph):
    n = len(graph)    
    cycle = find_cycle(graph, 0)
    print(graph)
    print(cycle)
    while True:
        for i, node in enumerate(cycle):
            if len(graph[node]) != 0:
                cycle = append_cycle(graph, cycle, node)
                print(graph)
                break
        if len(cycle) == n + 1:
            break
    print(cycle)

In [ ]:
g = [0] * 10
g[0] = [3]
g[1] = [0]
g[2] = [1,6]
g[3] = [2]
g[4] = [2]
g[5] = [4]
g[6] = [5,8]
g[7] = [9]
g[8] = [7]
g[9] = [6]
print(g)

eulerian_cycle(g)

[[3], [0], [1, 6], [2], [2], [4], [5, 8], [9], [7], [6]]
[[], [], [6], [], [2], [4], [5, 8], [9], [7], [6]]
[3, 2, 1, 0]
[6, 5, 4, 2]
[3, 2, 6, 5, 4, 2, 1, 0]
[[], [], [], [], [], [], [8], [9], [7], [6]]
[8, 7, 9, 6]
[3, 2, 6, 8, 7, 9, 6, 5, 4, 2, 1, 0]
[[], [], [], [], [], [], [], [], [], []]


In [ ]:
6->8->7->9->6->5->4->2->1->0->3->2->6